# Transformer Architecture

본 실습에서는 트랜스포머 아키텍처에 대해 자세히 살펴 보도록 하겠습니다.  
트랜스포머 인코더를 구성하는 다양한 블록들이 어떻게 구현되는지, 어떤 텐서들이 입출력되는지 확인해 본 다음,  
마지막으로 트랜스포머 인코더를 이용하여 Sequence Classification을 위한 Task Head를 구현해 보겠습니다.

## 0. Setup

In [1]:
import os
os.environ['HF_HOME'] = 'D:/HF/cache'
os.environ['HF_DATASETS'] = 'D:/HF/datasets'
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = "1"

In [2]:
from IPython.display import display
from tqdm.notebook import tqdm as notebook_tqdm

In [ ]:
# !pip install --user bertviz==1.4.1
# !pip install --user transformers==4.51.1

In [3]:
!nvidia-smi

Thu Jun 26 11:05:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 553.62                 Driver Version: 553.62         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q               WDDM  |   00000002:00:00.0 Off |                    0 |
| N/A    0C    P8             N/A /  N/A  |    1374MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 1. Transformer Attention Viewer

**`BertViz`** 라이브러리는 Transformer 모델에서 Attention의 다양한 측면을 시각화하는 함수를 제공합니다.  
(Head and Model Views, Neuron View)  
이때 모델은 **`output_attentions=True`** 로 설정해야 합니다.

In [4]:
from bertviz import head_view
from transformers import AutoModel, AutoTokenizer

model_id = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, output_attentions=True, attn_implementation="eager")

sentence_a = "time flies like an arrow"
sentence_b = "fruit flies like a banana"

viz_inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt')
attention = model(**viz_inputs).attentions
sentence_b_start = (viz_inputs.token_type_ids == 0).sum(dim=1)
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])

head_view(attention, tokens, sentence_b_start, heads=[8])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

<IPython.core.display.Javascript object>

In [5]:
text = "time flies like an arrow"
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=True)
inputs.input_ids

tensor([[  101,  2051, 10029,  2066,  2019,  8612,   102]])

## 2. Transformer Blocks

### 2.1 Token Embedding

Embedding Layer Output: [Batch Size, Sequence Length, Hidden Dimension]

In [6]:
from torch import nn
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_id)
token_embedding = nn.Embedding(config.vocab_size, config.hidden_size)
token_embedding

Embedding(30522, 768)

In [7]:
inputs_embeddings = token_embedding(inputs.input_ids)
inputs_embeddings.size()

torch.Size([1, 7, 768])

### 2.2 Position Embedding

논문에서는 Positional Encoding 방법을 사용하였지만, 본 예제에서는 Position Embedding을 사용합니다. 이 방법은 훈련 데이터가 충분할 때 효과적인 방법입니다. 토큰 임베딩과 동일한 방법으로 토큰 임베딩과 달리 입력이 Token ID가 아닌 Position ID 값을 사용합니다. 이를 통해 학습하는 동안 토큰 위치를 인코딩하는 효과적인 방법을 학습합니다.

In [8]:
import torch

class Embeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout()

    def forward(self, input_ids):
        # Sequence Length
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long).unsqueeze(0)
        # Token Embeddings & Position Embeddings
        token_embeddings = self.token_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        # Token Embeddings + Position Embeddings
        embeddings = token_embeddings + position_embeddings
        embeddings = self.layer_norm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

In [9]:
embedding_layer = Embeddings(config)
embedding_layer(inputs.input_ids).size()

torch.Size([1, 7, 768])

### 2.3 Multi-Head Attention

**Scaled Dot Product Attention** 을 구현합니다.  
Query, Key에 대한 Dot Product 연산, dim_k 기반 Scaling Factor를 곱해 정규화, SoftMax 함수 적용 순으로 이루어집니다.

In [10]:
seq_len = inputs.input_ids.size(-1)
mask = torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0)
mask[0]

tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1.]])

In [11]:
from math import sqrt
import torch.nn.functional as F

def scaled_dot_product_attention(query, key, value, mask=None):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float("-inf"))
    weights = F.softmax(scores, dim=-1)
    return weights.bmm(value)

**Attention Head**  
기본적인 **`Scaled Dot Product Attention`** 을 적용합니다.

In [12]:
class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)

    def forward(self, hidden_state):
        attn_outputs = scaled_dot_product_attention(
            self.q(hidden_state), self.k(hidden_state), self.v(hidden_state))
        return attn_outputs

**Multi-Head Attention**  
`head_dim`은 `embed_dim`을 `num_heads`로 나눈 값으로 설정됩니다.  
멀티헤드 각각의 출력(hidden_state)을 결합한 후에 최종 선형층을 한번 더 거치게 됩니다. 

In [13]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.hidden_size
        num_heads = config.num_attention_heads
        head_dim = embed_dim // num_heads
        self.heads = nn.ModuleList(
            [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)]
        )
        self.output_linear = nn.Linear(embed_dim, embed_dim)

    def forward(self, hidden_state):
        x = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
        x = self.output_linear(x)
        return x

In [14]:
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(inputs_embeddings)
attn_output.size()

torch.Size([1, 7, 768])

### 2.4 Feed Forward Layer

두개의 층으로 구성된 완전 연결 신경망입니다. 각 임베딩을 독립적으로 처리합니다.  
`nn.Linear` 피드 포워드 층은 일반적으로 `(batch_size, seq_len, hidden_dim)` 크기의 텐서를 전달하면 배치와 시퀀스의 모든 토큰 임베딩에 이 피드 포워드 층이 독립적으로 적용됩니다.

In [15]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.linear_1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.linear_2 = nn.Linear(config.intermediate_size, config.hidden_size)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.gelu(x)
        x = self.linear_2(x)
        x = self.dropout(x)
        return x

In [16]:
feed_forward = FeedForward(config)
ff_outputs = feed_forward(attn_output)
ff_outputs.size()

torch.Size([1, 7, 768])

## 3. Transformer Encoder

Layer Normalization 1 → Multi-Head Attention → Layer Normalization 2 → Feed Forward 순으로 구성합니다.

In [17]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer_norm_1 = nn.LayerNorm(config.hidden_size)
        self.layer_norm_2 = nn.LayerNorm(config.hidden_size)
        self.attention = MultiHeadAttention(config)
        self.feed_forward = FeedForward(config)

    def forward(self, x):
        # Layer Normalization-1
        hidden_state = self.layer_norm_1(x)
        # Skip Connection + Attention
        x = x + self.attention(hidden_state)
        # Skip Connection + Feed Forward (Layer Normalization-2)
        x = x + self.feed_forward(self.layer_norm_2(x))
        return x

In [18]:
encoder_layer = TransformerEncoderLayer(config)
inputs_embeddings.shape, encoder_layer(inputs_embeddings).size()

(torch.Size([1, 7, 768]), torch.Size([1, 7, 768]))

**`num_hidden_layers`** 만큼 TransformerEncoderLayer를 생성합니다.

In [19]:
class TransformerEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embeddings = Embeddings(config)
        self.layers = nn.ModuleList([TransformerEncoderLayer(config)
                                     for _ in range(config.num_hidden_layers)])

    def forward(self, x):
        x = self.embeddings(x)
        for layer in self.layers:
            x = layer(x)
        return x

In [20]:
encoder = TransformerEncoder(config)
encoder(inputs.input_ids).size()

torch.Size([1, 7, 768])

## 4. Task Head for Sequence Classification

In [21]:
class TransformerForSequenceClassification(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = TransformerEncoder(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, x):
        x = self.encoder(x)[:, 0, :] # Hidden State of [CLS] Token
        x = self.dropout(x)
        x = self.classifier(x)
        return x

In [22]:
config.num_labels = 6
encoder_classifier = TransformerForSequenceClassification(config)
encoder_classifier(inputs.input_ids).size()

torch.Size([1, 6])

- Ref. https://github.com/rickiepark/nlp-with-transformers